In [1]:
from os import path, listdir, mkdir
from os.path import join
from torch_geometric.data import HeteroData
import json

/home/ma952/.conda/envs/sets/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
# check if gpu is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
data_path = "../unarXiv"

In [4]:
def read_jsonl(file_path):
    with open(file_path, "r") as f:
        data = [json.loads(line) for line in f]
    return data

In [5]:
import pandas as pd

In [6]:
import os
os.getcwd()

'/home/ma952/Sets/2023sp_cs6850_network/WMRRank'

In [7]:
papers = read_jsonl(join(data_path, str(folder), files))
for paper in papers: 
    metadata = paper.get('metadata', {})
    if metadata.get('authors_parsed'):
        print(metadata['authors_parsed'])

NameError: name 'folder' is not defined

PROCESSING

In [8]:
import warnings
from tqdm import tqdm

In [9]:
warnings.filterwarnings('ignore')
total_papers = 0
authors = []
papers = []
paper_info = pd.DataFrame(columns=['paper_id', 'title', 'authors', 'cited_paper_ids'])
for folder in listdir(data_path):
    if folder in ['00', '01']:
        print(folder, os.path.join(data_path, str(folder)))
        if folder[0] == "9":
            year = "19" + folder
        else:
            year = "20" + folder
        for files in tqdm(listdir(join(data_path, str(folder)))):
            papers_read = read_jsonl(join(data_path, str(folder), files))
            total_papers += len(papers_read)
            for paper in papers_read:
                cited_paper_ids = []
                author_names = []
                metadata = paper.get('metadata', {})
                if metadata.get('title') and metadata.get('authors_parsed'):
                    paper_title = metadata['title']
                    paper_id = paper["paper_id"]
                    # print(paper_id)
                    papers.append(paper_id) # 1 Paper
                    for cited_paper_id, cited_paper_info in paper["bib_entries"].items():
                        if cited_paper_info.get('contained_arXiv_ids'):
                            # print(len(cited_paper_info['contained_arXiv_ids']))
                            cited_paper_ids.append(cited_paper_info['contained_arXiv_ids'][0].get('id')) # 1 Paper at a time 
                            # print(cited_paper_info['contained_arXiv_ids'][0].get('id'))
                    papers.extend(cited_paper_ids)
                    if metadata.get('authors_parsed'):
                        authors_parsed = metadata['authors_parsed']
                        # Consider only top 5 authors
                        for author in authors_parsed[:5]:
                            author = author[:2]
                            author_parsed = " ".join(author)
                            author_names.append(author_parsed)
                        authors.extend(author_names) # Multiple authors at a time 
                    paper_info = paper_info.append({'paper_id': paper_id, 'title': paper_title, 'authors': author_names, 'cited_paper_ids': cited_paper_ids, 'year': year}, ignore_index=True)

authors_pd = pd.DataFrame(authors)
authors_pd = authors_pd.drop_duplicates()
papers_pd = pd.DataFrame(papers)
papers_pd = papers_pd.drop_duplicates()

00 ../unarXiv/00


100%|███████████████████████████████████████████| 12/12 [02:25<00:00, 12.17s/it]


01 ../unarXiv/01


100%|███████████████████████████████████████████| 12/12 [05:50<00:00, 29.24s/it]


In [ ]:
paper_info.to_csv('paper_info.csv', index=False)
papers_pd.to_csv('papers.csv', index=False, header=False)
authors_pd.to_csv('authors.csv', index=False, header=False)

In [ ]:
import networkx as nx

In [10]:
print(authors_pd)
print(papers_pd)

                                 0
0                  Bittner Eric R.
1                    Wiseman H. M.
2                        Molmer K.
3                           You L.
4                  Sudbery Anthony
...                            ...
141079                   Camus Ph.
141083  Collaboration the Archeops
141086                 Shlosman I.
141087               Mahoney T. J.
141097              Allen Glenn E.

[56482 rows x 1 columns]
                      0
0      quant-ph/0001119
1      quant-ph/0001118
2      quant-ph/0001117
3      quant-ph/0001116
4      quant-ph/0001114
...                 ...
58651  astro-ph/0112005
58652  astro-ph/0112004
58653  astro-ph/0112003
58654  astro-ph/0112002
58655  astro-ph/0112001

[57933 rows x 1 columns]


In [ ]:
# print("Paper Info:")
# print()
# print(paper_info.head())
# print(total_papers)
# authors = list(authors)
# papers = list(papers)
# print(papers)
# print(len(paper_info))

In [ ]:
# for i,v in paper_info.iterrows():
#     print(v['paper_id'])

In [11]:
unique_paper_id = papers_pd[0].unique()
unique_paper_id = pd.DataFrame(data={
    'paperId': unique_paper_id,
    'mappedID': pd.RangeIndex(len(unique_paper_id)),
})
print("Mapping of paper IDs to consecutive values:")
print("==========================================")
print(unique_paper_id.head())
print()
unique_paper_id.to_csv('unique_paper_id.csv', index=False, header=False)

# for i,v in unique_paper_id.iterrows():
#     print(v)

# unique_author_id = authors_pd[0].unique()
# unique_author_id = pd.DataFrame(data={
#     'authorId': unique_author_id,
#     'mappedID': pd.RangeIndex(len(unique_author_id)),
# })
# print("Mapping of author IDs to consecutive values, remember each author in a year is treated as a new entity:")
# print("==========================================")
# print(unique_author_id.head())
# print()

Mapping of paper IDs to consecutive values:
            paperId  mappedID
0  quant-ph/0001119         0
1  quant-ph/0001118         1
2  quant-ph/0001117         2
3  quant-ph/0001116         3
4  quant-ph/0001114         4



Computing total entities

In [12]:
paper_info_2 = paper_info.explode('authors')
# Computing unique total entities by treating authors in new years as seperate entities
paper_info_2['authors_year'] = paper_info_2['authors'].astype(str) + '_' + paper_info_2['year'].astype(str)

total_author_entities = len(paper_info_2['authors_year'].unique())
unique_author_year_id = paper_info_2['authors_year'].unique()

unique_author_year_id = pd.DataFrame(data={
    'authorId': unique_author_year_id,
    'mappedID': pd.RangeIndex(total_author_entities),
})

print("Total number of unique author entities: ", total_author_entities)

# Using this to create the edges of the heterograph
paper_info_3 = paper_info_2.explode('cited_paper_ids')

Total number of unique author entities:  74109


In [13]:
# print(unique_paper_id)
paper_id_info = pd.merge(paper_info_3['paper_id'], unique_paper_id, left_on='paper_id', right_on='paperId', how='left')
# print(paper_id_info)
paper_id_info = torch.from_numpy(paper_id_info['mappedID'].values)

paper_cite_info = pd.merge(paper_info_3['cited_paper_ids'], unique_paper_id, left_on='cited_paper_ids', right_on='paperId', how='left')
paper_cite_info = torch.from_numpy(paper_cite_info['mappedID'].values)

paper_author_year_info = pd.merge(paper_info_3['authors_year'], unique_author_year_id, left_on='authors_year', right_on='authorId', how='left')
paper_author_year_info = torch.from_numpy(paper_author_year_info['mappedID'].values)

# Construct the edges:
paper_author_edge_index = torch.stack([paper_id_info, paper_author_year_info], dim=0)
paper_cite_edge_index = torch.stack([paper_id_info, paper_cite_info], dim=0)

print()
print("Edge indices:")
print("=============")
print("Final edge indices pointing from Paper to author:")
print("------------------------")
print(paper_author_edge_index)
print()
print("Final edge indices pointing from Paper to cited papers:")
print("------------------------")
print(paper_cite_edge_index)
print()


Edge indices:
Final edge indices pointing from Paper to author:
------------------------
tensor([[    0,     1,     1,  ..., 57931, 57932, 57932],
        [    0,     1,     2,  ..., 58682, 39630, 40015]])

Final edge indices pointing from Paper to cited papers:
------------------------
tensor([[0.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.7931e+04, 5.7932e+04,
         5.7932e+04],
        [       nan,        nan,        nan,  ...,        nan,        nan,
                nan]], dtype=torch.float64)



In [14]:
import torch_geometric.transforms as T
data = HeteroData()

data["paper"].node_id = torch.arange(len(unique_paper_id))
data["author"].node_id = torch.arange(len(unique_author_year_id))


data["paper", "written_by", "author"].edge_index = paper_author_edge_index
data["paper", "cites", "paper"].edge_index = paper_cite_edge_index

print(data)

HeteroData(
  paper={ node_id=[57933] },
  author={ node_id=[74109] },
  (paper, written_by, author)={ edge_index=[2, 144246] },
  (paper, cites, paper)={ edge_index=[2, 144246] }
)


START OF WMR RANK

In [15]:
import numpy as np

In [16]:
total_author_entities

74109

In [207]:
MU = { 1991: 0.0, 1992: 0.0, 1993: 0.0, 1994: 0.0, 1995: 0.0, 1996: 0.0, 1997: 0.0, 1998: 0.0, 1999: 0.0,
       2000: 0.1, 2001: 0.1, 2002: 0.1, 2003: 0.1, 2004: 0.1, 2005: 0.1, 2006: 0.2, 2007: 0.2, 2008: 0.2, 
       2009: 0.2, 2010: 0.3, 2011: 0.4, 2012: 0.5, 2013: 0.5, 2014: 0.5, 2015: 0.5, 2016: 0.6, 2017: 0.6, 
       2018: 0.7, 2019: 0.8, 2020: 0.8, 2021: 0.8, 2022: 0.8, 2023: 0.9 }

class WMR_Rank:
    
    def __init__(self, heteroData, paper_info_df, range_start, range_end, total_author_entities):
        self.yr_start = range_start
        self.yr_end = range_end
        self.paper_info = paper_info_df
        self.heteroData = heteroData
        self.total_author_entities = total_author_entities
        
    def get_mapped_author_year(self, author_year):
        return unique_author_year_id[unique_author_year_id['authorId'] == author_year]['mappedID'].values[0]
    
    def get_mapped_paper(self, paper):
        return unique_paper_id[unique_paper_id['paperId'] == paper]['mappedID'].values[0]
                
    def order(self, paper, author):
        paper_df = self.paper_info
        paper_df = paper_df[paper_df['paper_id'] == paper]
        authors = list(paper_df['authors'])
        return authors.index(author) + 1

    def pnai(self, AS, paper_info, author_year, ta):
        author = '_'.join(author_year.split('_')[:-1])
        current_year = int(author_year.split('_')[-1])
        author_perf = torch.zeros(ta).type(torch.float64)
        for i in range(ta):
            author_pass = author + '_' + str(current_year - i - 1)
            if author_pass in paper_info['authors_year'].values:
                author_perf[i] = AS[self.get_mapped_author_year(author_pass)]
        return author_perf
    
    def find_papers_citing_paper(self, paper):
        edge_index = self.heteroData['paper', 'cites', 'paper'].edge_index
        paper_mapped = unique_paper_id[unique_paper_id['paperId'] == paper]['mappedID'].unique()[0]
        papers_cited_mapped = edge_index[0][edge_index[1] == paper_mapped]
        papers_cited_mapped = list(set(papers_cited_mapped.numpy().tolist()))
        if paper_mapped in papers_cited_mapped:
            papers_cited_mapped.remove(paper_mapped)
        papers_cited_mapped = unique_paper_id[unique_paper_id['mappedID'].isin(papers_cited_mapped)]['paperId'].values
        return papers_cited_mapped
    
    def recent_papers_citing_paper(self, paper, current_year, l):
        paper_df = self.paper_info
        citing_papers = self.find_papers_citing_paper(paper)
        years_citing_papers = []
        years_citing_papers = paper_df[paper_df['paper_id'].isin(citing_papers)]['year'].values
        years_citing_papers = list(map(int, years_citing_papers))
        recent_papers = [citing_papers[i] for i in range(len(citing_papers)) if ((current_year - years_citing_papers[i]) <= l)]
        return recent_papers
    
    def T_pi(self, paper, current_year, sigma1):
        paper_df = self.paper_info
        publish_year = int(self.paper_published_year(paper))
        evaluation_year = current_year
        l = int(evaluation_year) - publish_year
        recent_citing_papers = self.recent_papers_citing_paper(paper, current_year, l)
        years_citing_papers = []
        years_citing_papers = paper_df[paper_df['paper_id'].isin(recent_citing_papers)]['year'].values
        years_citing_papers = list(years_citing_papers)
        if len(recent_citing_papers) < l:
            years_citing_papers.extend([publish_year] * (len(recent_citing_papers) - l))
        years_citing_papers = list(map(int, years_citing_papers))
        years_citing_papers = np.array(years_citing_papers)
        T_avg = np.mean(years_citing_papers)
        T_pi = np.exp(-sigma1 * (current_year - T_avg))/ (current_year - T_avg)
        return T_pi
    
    def T_ai(self, author_year, sigma2, m = 10):
        paper_df = self.paper_info
        papers = self.get_all_papers_by_author_year(author_year)
        citing_papers = []
        for i in range(len(papers)):
            citing_papers.extend(self.find_papers_citing_paper(papers[i]))
        citing_papers = list(set(citing_papers))
        years_citing_papers = []
        years_citing_papers = list(map(int, paper_df[paper_df['paper_id'].isin(citing_papers)]['year'].values))
        years_citing_papers = np.array(years_citing_papers)
        years_citing_papers = np.sort(years_citing_papers)
        years_citing_papers = years_citing_papers[::-1]
        year = int(author_year.split('_')[1])
        if years_citing_papers.shape[0] > m:
            years_citing_papers = years_citing_papers[:int(m)]
        else:
            years_citing_papers = list(years_citing_papers)
            years_citing_papers.extend([year] * (int(m) - len(years_citing_papers)))
        T_avg = np.mean(years_citing_papers)
        T_ai = np.exp(-sigma2 * (year - T_avg))/ (year - T_avg)
        return T_ai       
        
    def paper_paper_edge_exists(self, paper_i, paper_j):
        paper_i_mapped = unique_paper_id[unique_paper_id['paperId'] == paper_i]['mappedID'].values[0]
        paper_j_mapped = unique_paper_id[unique_paper_id['paperId'] == paper_j]['mappedID'].values[0]
        edge_index = self.heteroData['paper', 'cites', 'paper'].edge_index
        has_edge = torch.where((edge_index[0] == paper_j_mapped) & (edge_index[1] == paper_i_mapped))
        if len(has_edge[0]) > 0:
            return True
        else:
            return False
        
    # def paper_author_edge_exists(self, paper, author):
    #     paper_mapped = unique_paper_id[unique_paper_id['paperId'] == paper]['mappedID'].values[0]
    #     author_mapped = unique_author_id[unique_author_id['authorId'] == author]['mappedID'].values[0]
    #     edge_index = self.heteroData['paper', 'written_by', 'author'].edge_index
    #     has_edge = torch.where((edge_index[0] == paper_mapped) & (edge_index[1] == author_mapped))    
    #     if len(has_edge[0]) > 0:
    #         return True
    #     else:
    #         return False
      
    def get_all_papers_by_author_year(self, author_year):
        author_mapped = self.get_mapped_author_year(author_year)
        edge_index = self.heteroData['paper', 'written_by', 'author'].edge_index
        papers = edge_index[0][edge_index[1] == author_mapped]
        papers = papers.numpy().tolist()
        papers = unique_paper_id[unique_paper_id['mappedID'].isin(papers)]['paperId'].values
        papers = list(set(papers))
        return papers
        
    def get_papers_cited_by_paper(self, paper):
        paper_mapped = unique_paper_id[unique_paper_id['paperId'] == paper]['mappedID'].values[0]
        edge_index = self.heteroData['paper', 'cites', 'paper'].edge_index
        papers = edge_index[1][edge_index[0] == paper_mapped]
        papers = papers.numpy().tolist()
        if paper_mapped in papers:
            papers.remove(paper_mapped)
        papers = unique_paper_id[unique_paper_id['mappedID'].isin(papers)]['paperId'].values
        papers = list(set(papers))
        return papers
    
    def get_all_authors_of_paper(self, paper):
        paper_mapped = unique_paper_id[unique_paper_id['paperId'] == paper]['mappedID'].values[0]
        edge_index = self.heteroData['paper', 'written_by', 'author'].edge_index
        authors = edge_index[1][edge_index[0] == paper_mapped]
        authors = authors.numpy().tolist()
        authors = unique_author_year_id[unique_author_year_id['mappedID'].isin(authors)]['authorId'].values
        authors = list(set(authors))
        return authors
    
    def get_all_coauthors_and_citing_authors(self, author_year):
        papers_by_author = self.get_all_papers_by_author_year(author_year)
        coauthors = []
        for i in range(len(papers_by_author)):
            authors_paper = self.get_all_authors_of_paper(papers_by_author[i])
            coauthors.extend(authors_paper)
        coauthors = list(set(coauthors))
        authors_that_cited = []
        for i in range(len(papers_by_author)):
            citing_papers = self.find_papers_citing_paper(papers_by_author[i])
            for j in range(len(citing_papers)):
                authors_that_cited.extend(self.get_all_authors_of_paper(citing_papers[j]))
                
        authors_that_cited = list(set(authors_that_cited))
        final_authors = list(set(coauthors + authors_that_cited))
        if author_year in final_authors:
            final_authors.remove(author_year)
        return final_authors
    
    def W_ca_raw(self, author_m, author_n, paper_i, paper_j):
        author_m = author_m.split('_')[0]
        author_n = author_n.split('_')[0]
        return self.order(paper_i, author_m) * self.order(paper_j, author_n)

    def W_ca(self, author_m, author_n, paper_i, paper_j):
        num = self.W_ca_raw(author_m, author_n, paper_i, paper_j)
        authors_k = self.get_all_authors_of_paper(paper_i)
        authors_l = self.get_all_authors_of_paper(paper_j)
        denom = 0.0
        for k in range(len(authors_k)):
            for l in range(len(authors_l)):
                if authors_k[k] != authors_l[l]:
                    denom = denom + self.W_ca_raw(authors_k[k], authors_l[l], paper_i, paper_j)
        return num/denom

    def W_ca_total(self, author_m, author_n):
        p_i = self.get_all_papers_by_author_year(author_m)
        p_j = self.get_all_papers_by_author_year(author_n)
        sum_final = 0.0
        for i in range(len(p_i)):
            for j in range(len(p_j)):
                if self.paper_paper_edge_exists(p_i[i], p_j[j]) and p_i[i] != p_j[j]:
                    sum_final = sum_final + self.W_ca(author_m, author_n, p_i[i], p_j[j])
        return sum_final
    
    def W_coa_raw(self, author_i, author_j, paper):
        author_i = author_i.split('_')[0]
        author_j = author_j.split('_')[0]
        # print("Author i: ", author_i, " Author j: ", author_j, " Paper: ", paper)
        return 1. / (self.order(paper, author_i) * self.order(paper, author_j))
    
    def W_coa(self, author_i, author_j, p):
        num = self.W_coa_raw(author_i, author_j, p)
        denom = 0.0
        authors = self.get_all_authors_of_paper(p)
        for i in range(len(authors)):
            for j in range(i+1, len(authors)):
                denom = denom + self.W_coa_raw(authors[i], authors[j], p)
        return num/denom
    
    def W_coa_total(self, author_i, author_j):
        papers_i = self.get_all_papers_by_author_year(author_i)
        papers_j = self.get_all_papers_by_author_year(author_j)
        papers = list(set(papers_i) & set(papers_j))
        sum_final = 0.0
        for i in range(len(papers)):
            sum_final = sum_final + self.W_coa(author_i, author_j, papers[i])
        return sum_final
    
    def W_aa(self, author_i, author_j):
        return self.W_ca_total(author_i, author_j) + self.W_coa_total(author_i, author_j)        
    
    def W_pa(self, paper, author_year):
        num_authors = len(self.get_all_authors_of_paper(paper))
        author = author_year.split('_')[0]
        position_author = self.order(paper, author)
        return (2**(num_authors - position_author)) / (2**num_authors - 1) 
    
    def W_pp(self, p_i, p_j):
        if self.paper_paper_edge_exists(p_i, p_j):
            return 1
        else:
            return 0
        
    def paper_published_year(self, paper):
        if paper_info[paper_info['paper_id'] == paper]['year'].values:
            return paper_info[paper_info['paper_id'] == paper]['year'].values[0]
        else:
            year = paper.split('/')[1][:2]
            if int(year) > 20:
                return '19' + year
            else:
                return '20' + year
            
        
    # alpha = 0.70, beta = 0.60, lmbda = 0.85,  sigma_1 = 0.5, sigma_2 = 1.0, mu = step_function
    def algo(self, current_year, VP, VA, alpha, beta, sigma1, sigma2, lmbda, epsilon, ta=2):
        # Assumption
        mu = MU
        
        PS = torch.zeros(11, len(VP)) # Capped at 10 years
        PS[0] = torch.ones(len(VP)) / len(VP)
        
        AS = torch.zeros(11, len(VA))
        AS[0] = torch.ones(len(VA)) / len(VA)
        
        pas = torch.ones(len(VA))
        t = -1 # Represents the current_year - 1 in terms of symbolic notation
        # 1 represents first analysis year
        delta = 2 * epsilon
        while delta > epsilon and tqdm(t<11):
            t = t + 1 # First iteration 0
            for author_year in tqdm(VA):
                author_array = self.pnai(AS[t], self.paper_info, author_year, ta)
                author_year_mapped = self.get_mapped_author_year(author_year)
                pas[author_year_mapped] = (1 / (ta + 1)) * (torch.sum(author_array) + AS[t][author_year_mapped])
                
            sum_temp = 0
            temp = torch.zeros(len(VA))
            for author_year in tqdm(VA):
                year = author_year.split('_')[1]
                author_year_mapped = self.get_mapped_author_year(author_year)
                for paper in self.get_all_papers_by_author_year(author_year):
                    paper_mapped = self.get_mapped_paper(paper)
                    sum_temp = sum_temp + (self.W_pa(paper, author_year) * PS[t][paper_mapped])
                temp[author_year_mapped] = beta * sum_temp
                temp_compute = 0
                for author_found in self.get_all_coauthors_and_citing_authors(author_year):
                    author_found_mapped = self.get_mapped_author_year(author_found)
                    temp_compute = temp_compute + self.W_aa(author_found, author_year) * temp[author_found_mapped] + mu[int(year)] * temp[author_year_mapped]
                temp[author_year_mapped] = temp_compute
                AS[t+1][author_year_mapped] = lmbda * self.T_ai(author_year, current_year, sigma2) * temp[author_year_mapped] + (1 - lmbda) * (1/len(VA))

            temp = torch.zeros(len(VP))   
            for paper in tqdm(VP):
                sum_authors = 0
                paper_year = int(self.paper_published_year(paper))
                paper_mapped = self.get_mapped_paper(paper)
                for author_year in self.get_all_authors_of_paper(paper):
                    author_year_mapped = self.get_mapped_author_year(author_year)
                    sum_authors = sum_authors + (self.W_pa(paper, author_year) * pas[author_year_mapped])
                temp[paper_mapped] = alpha * sum_authors
                for paper_citing in self.find_papers_citing_paper(paper):
                    paper_citing_mapped = self.get_mapped_paper(paper_citing)
                    temp[paper_mapped] = temp[paper_citing_mapped] + mu[paper_year] * temp[paper_mapped]
                PS[t+1][paper_mapped] = lmbda * self.T_pi(paper, current_year, sigma1) * temp[paper_mapped] + (1 - lmbda) * (1/len(VP))
            
            delta = 0
            delta = delta + torch.sum(abs(PS[t+1] - PS[t]))
            delta = delta + torch.sum(abs(AS[t+1] - AS[t]))
    
        return AS, PS

In [208]:
wmr = WMR_Rank(data, paper_info_2, 2000, 2001, total_author_entities)

In [178]:
wmr.order('quant-ph/0001014', 'Pittenger Arthur O.')

1

In [20]:
wmr.W_pp('astro-ph/0202187', 'hep-ph/0004227')

['astro-ph/0202187' 'hep-ph/9809265' 'hep-ph/9710460' 'hep-ph/0007176'
 'hep-ph/0101307' 'hep-ph/0107014' 'hep-ph/9911315' 'hep-ph/0012345'
 'hep-ph/0202058' 'nucl-ex/0204009' 'hep-ph/9608308' 'hep-ph/9904310'
 'hep-ph/0202030']


1

In [179]:
wmr.find_papers_citing_paper('astro-ph/0202187')

array(['hep-ph/0004227'], dtype=object)

In [182]:
papers = wmr.get_all_papers_by_author_year('Bonnell Ian A._2001')
print(papers)
wmr.get_mapped_paper(papers[0])

['astro-ph/0111275', 'astro-ph/0112001', 'astro-ph/0101085']


55192

In [183]:
print(paper_info_2)

               paper_id                                              title  \
0      quant-ph/0001119  Quantum tunneling dynamics using hydrodynamic ...   
1      quant-ph/0001118  Induced coherence with and without induced emi...   
1      quant-ph/0001118  Induced coherence with and without induced emi...   
2      quant-ph/0001117  Motional effects of single trapped atomic/ioni...   
3      quant-ph/0001116     On local invariants of pure three-qubit states   
...                 ...                                                ...   
57103  astro-ph/0112002  The Chandra Deep Field North Survey. IX. Exten...   
57103  astro-ph/0112002  The Chandra Deep Field North Survey. IX. Exten...   
57103  astro-ph/0112002  The Chandra Deep Field North Survey. IX. Exten...   
57104  astro-ph/0112001  The brown dwarf desert as a consequence of orb...   
57104  astro-ph/0112001  The brown dwarf desert as a consequence of orb...   

                   authors cited_paper_ids  year              a

In [23]:
print(authors_pd.values.tolist())

[['Bittner Eric R.'], ['Wiseman H. M.'], ['Molmer K.'], ['You L.'], ['Sudbery Anthony'], ['Wootters William K.'], ['Unnikrishnan C. S.'], ['Yeung Wai Bong'], ['Pittenger Arthur O.'], ['Rubin Morton H.'], ['Pinamonti N.'], ['Toller M.'], ['Clifton Rob'], ['Halvorson Hans'], ['Farhi Edward'], ['Goldstone Jeffrey'], ['Gutmann Sam'], ['Sipser Michael'], ['Song David D.'], ['Hardy Lucien'], ['Appleby D. M.'], ['Huelga S. F.'], ['Plenio M. B.'], ['Jung C.'], ['Leyvraz F.'], ['Seligman T. H.'], ['Rangelov Josiph Mladenov'], ['Yaremchuk O.'], ['Neumaier Arnold'], ['Viola L.'], ['Fortunato E. M.'], ['Lloyd S.'], ['Tseng C. -H.'], ['Cory D. G.'], ['Fleischhauer M.'], ['Lukin M. D.'], ['Griffiths Robert B.'], ['Kim Ilki'], ['Mahler Guenter'], ['Carteret H A'], ['Sudbery A'], ['Albeverio Sergio'], ['Dabrowski Ludwik'], ['Fei Shao-Ming'], ['Buttler W. T.'], ['Hughes R. J.'], ['Lamoreaux S. K.'], ['Morgan G. L.'], ['Nordholt J. E.'], ['Smith Adam D.'], ['Haba Z.'], ['Cohen Oliver'], ['Brun Todd A.']

In [32]:
# alpha = 0.70, beta = 0.60, lmbda = 0.85,  sigma_1 = 0.5, sigma_2 = 1.0, mu = [0.2, 0.8, 0.2], VA = list of authors_pd, VP = list of papers_pd
authors_pd_list = list(unique_author_year_id['authorId'])
papers_pd_list = list(papers_pd[0])
print(authors_pd_list, papers_pd_list)

['Bittner Eric R._2000', 'Wiseman H. M._2000', 'Molmer K._2000', 'You L._2000', 'Sudbery Anthony_2000', 'Wootters William K._2000', 'Unnikrishnan C. S._2000', 'Yeung Wai Bong_2000', 'Pittenger Arthur O._2000', 'Rubin Morton H._2000', 'Pinamonti N._2000', 'Toller M._2000', 'Clifton Rob_2000', 'Halvorson Hans_2000', 'Farhi Edward_2000', 'Goldstone Jeffrey_2000', 'Gutmann Sam_2000', 'Sipser Michael_2000', 'Song David D._2000', 'Hardy Lucien_2000', 'Appleby D. M._2000', 'Huelga S. F._2000', 'Plenio M. B._2000', 'Jung C._2000', 'Leyvraz F._2000', 'Seligman T. H._2000', 'Rangelov Josiph Mladenov_2000', 'Yaremchuk O._2000', 'Neumaier Arnold_2000', 'Viola L._2000', 'Fortunato E. M._2000', 'Lloyd S._2000', 'Tseng C. -H._2000', 'Cory D. G._2000', 'Fleischhauer M._2000', 'Lukin M. D._2000', 'Griffiths Robert B._2000', 'Kim Ilki_2000', 'Mahler Guenter_2000', 'Carteret H A_2000', 'Sudbery A_2000', 'Albeverio Sergio_2000', 'Dabrowski Ludwik_2000', 'Fei Shao-Ming_2000', 'Buttler W. T._2000', 'Hughes 

In [ ]:
AS, PS = wmr.algo(2001, papers_pd_list, authors_pd_list, 0.70, 0.60, 0.5, 1.0, 0.85, 1e-6, 2)